In [ ]:
%matplotlib inline
#%matplotlib notebook

For high dpi displays.

In [ ]:
%config InlineBackend.figure_format = 'retina'

# 0. General note

- Loads in ruby spectrum .asc files.
- Fits ruby spectrum with 2 Lorentzians and a linear background.
- John Lazarz - 171205

# 1. General setup

In [ ]:
import os
import sys
import numpy as np
from scipy import constants as con
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import uncertainties as uct
from uncertainties import unumpy as unp
from lmfit import Model
from lmfit.models import LinearModel, LorentzianModel
import pandas as pd
from IPython.display import display
import burnman
import glob
import re

In [ ]:
if not os.path.exists('burnman') and os.path.exists('../burnman'):
    sys.path.insert(1, os.path.abspath('..'))

### What sample is this? - Where are the data folders?

In [ ]:
sample = 'Ruby'
T = 298.15

ruby_path = r'C:\data\Huber\Bausch\acet_ruby\ruby1'

if os.path.exists(os.path.abspath(ruby_path)):
    ruby_folder = os.path.abspath(ruby_path)
    print('Using Huber working directory.')
else:
    ruby_path = r'C:\data\Raman\Bausch\acet_ruby\ruby1'
    
    if os.path.exists(os.path.abspath(ruby_path)):
        ruby_folder = os.path.abspath(ruby_path)
        print('Using Raman working directory.')
    else:
        print('Directory does not exist.......')

ruby1_path = r'C:\data\Raman\Bausch\acet_ruby\ruby1'
ruby2_path = r'C:\data\Raman\Bausch\acet_ruby\ruby2'
ruby3_path = r'C:\data\Raman\Bausch\acet_ruby\ruby3'
ruby4_path = r'C:\data\Raman\Bausch\acet_ruby\ruby4'
ruby5_path = r'C:\data\Raman\Bausch\acet_ruby\ruby5'
ruby1v_path = r'C:\data\Raman\Bausch\acet_ruby\ruby1vert'
ruby2v_path = r'C:\data\Raman\Bausch\acet_ruby\ruby2vert'
ruby3v_path = r'C:\data\Raman\Bausch\acet_ruby\ruby3vert'
ruby4v_path = r'C:\data\Raman\Bausch\acet_ruby\ruby4vert'
ruby5v_path = r'C:\data\Raman\Bausch\acet_ruby\ruby5vert'

dirs = [ruby1_path,ruby2_path,ruby3_path,ruby4_path,ruby5_path,ruby1v_path,ruby2v_path,ruby3v_path,ruby4v_path,ruby5v_path]

# 2. Read data

Load in the .log files from directory as well as ruby .asc files from a folder named "ruby" within the directory.


In [ ]:
#idx_file = 0
#sample = logs[idx_file]
for i in dirs:
    os.chdir(i)
    rubys = glob.glob('*.asc')

# 3. Organize and fit data

First, we sort the ruby spectra into 2 lists. One list for 'table_rubys' and one list for 'sample_rubys'.

#### extract_ruby
In the 'extract_ruby' function we load the ruby spectrum from '.asc' files into lists. We then fit the data in the 'ruby_fit' funciton using the (Dawaele et al. 2008) equation to determine pressure. 

#### extract_cell
In the 'extract_cell' function we extract the cell information from the log file. This section searches through the log file and loads lattice parameters from the **last** "lsq" and the **last** "clsq" performed in Single in that log file. These lattice parameters, as well as standard deviations, are loaded into "cell" dictionary. 

Parameters collected from both "lsq" and "clsq": a, a_sigma, b, b_sigma, c, c_sigma, alpha, alpha_sigma, beta, beta_sigma ,gamma, gamma_sigma, volume, volume_sigma

To access parameters example:
 - To print "a" parameter from "lsq" function:
     - print(cell["lsq"]["a"])
 - To print "a_sigma" parameter from "clsq" function:
     - print(cell["clsq"]["a_sigma"])

In [ ]:
def calc_pressure(sample_lam):
    a = 1920.0
    b = 9.61
    r1 = sample_lam
    r1_o = 694.33
    p = (a/b)*(((r1/r1_o)**b)-1)
    return p

def normalize(y):
    z = (y-np.min(y))/(np.max(y)-np.min(y))
    return z

def ruby_fit(x,y_data,ruby):
    y = normalize(y_data)
    background  = LinearModel(prefix='lin_')
    pars = background.guess(y, x=x)

    r1 = LorentzianModel(prefix='r1_')
    pars.update(r1.make_params())
    pars['r1_center'].set(694.3)
    pars['r1_sigma'].set(0.000954)
    r2 = LorentzianModel(prefix='r2_')
    pars.update(r2.make_params())

    pars['r2_center'].set(692.9)
    pars['r2_sigma'].set(0.00135)
    mod = r1 + r2 + background
    init = mod.eval(pars, x=x)
    plt.plot(x, y)

    out = mod.fit(y, pars, x=x)

    comps = out.eval_components(x=x)

    print(out.fit_report(min_correl=0.5))

    plt.plot(x, out.best_fit, 'r-')
    plt.plot(x, comps['r1_'], 'b--')
    plt.plot(x, comps['r2_'], 'b--')
    plt.plot(x, comps['lin_'], 'k--')
    plt.title(ruby,fontweight='bold')
    plt.xlabel('Wavelength (nm)',fontweight='bold')
    plt.ylabel('Intensity (arb. units)',fontweight='bold')

    r1_center = str(out.params['r1_center']).split()[2]
    r1_center = float(r1_center.strip('value='))
    r1_esd = str(out.params['r1_center']).split()[4]
    r1_esd = float(r1_esd.strip(','))
    r2_center = str(out.params['r2_center']).split()[2]
    r2_center = float(r2_center.strip('value='))
    r2_esd = str(out.params['r2_center']).split()[4]
    r2_esd = float(r2_esd.strip(','))

    ruby_cent = [r1_center,r2_center]
    ruby_esd = [r1_esd,r2_esd]
    #print(ruby_cent)
    #print(ruby_esd)
    return ruby_cent

def extract_ruby(ruby):    
    with open(ruby, "r") as f:
        data = np.loadtxt(f)

    x = data[:,0]
    y = data[:,1]
    
    y_norm = normalize(y)
    return x,y_norm

count = 0
for i in dirs:
    print(i)
    os.chdir(i)
    
    rubys = glob.glob('*.asc')
        
    table_rubys = []
    sample_rubys = []
    
    for j in rubys:
        if re.search('table',j):
            table_rubys.append(j)
        else:
            sample_rubys.append(j)
            
    #table ruby wavelength
    table_lam = []
    for j in table_rubys:
        fig = plt.figure()
        x,y = extract_ruby(j)
        r1,r2 = ruby_fit(x,y,j)
        table_lam.append(r1)
    
    sample_lam = []
    for j in sample_rubys:
        fig = plt.figure()
        x,y = extract_ruby(j)
        r1,r2 = ruby_fit(x,y,j)
        sample_lam.append(r1)

    if count == 0:
        ruby1_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby1_ps.append(p)
    if count == 1:
        ruby2_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby2_ps.append(p)
    if count == 2:
        ruby3_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby3_ps.append(p)
    if count == 3:
        ruby4_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby4_ps.append(p)
    if count == 4:
        ruby5_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby5_ps.append(p)
    if count == 5:
        ruby1v_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby1v_ps.append(p)
    if count == 6:
        ruby2v_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby2v_ps.append(p)
    if count == 7:
        ruby3v_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby3v_ps.append(p)
    if count == 8:
        ruby4v_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby4v_ps.append(p)
    if count == 9:
        ruby5v_ps = []
        for k in range(len(sample_lam)):
            p = calc_pressure(sample_lam[k])
            ruby5v_ps.append(p)

    count += 1

In [ ]:
print('Ruby - 1: \n')
for i in range(len(ruby1_ps)): 
    print('P%s - %.3f GPa' % (i,ruby1_ps[i]))
    
print('\nRuby - 2: \n')
for i in range(len(ruby2_ps)): 
    print('P%s - %.3f GPa' % (i,ruby2_ps[i]))

print('\nRuby - 3: \n')
for i in range(len(ruby3_ps)): 
    print('P%s - %.3f GPa' % (i,ruby3_ps[i]))

print('\nRuby - 4: \n')
for i in range(len(ruby4_ps)): 
    print('P%s - %.3f GPa' % (i,ruby4_ps[i]))
    
print('\nRuby - 5: \n')
for i in range(len(ruby5_ps)): 
    print('P%s - %.3f GPa' % (i,ruby5_ps[i]))

print('Ruby - 1v: \n')
for i in range(len(ruby1v_ps)): 
    print('P%s - %.3f GPa' % (i,ruby1_ps[i]))
    
print('\nRuby - 2v: \n')
for i in range(len(ruby2v_ps)): 
    print('P%s - %.3f GPa' % (i,ruby2_ps[i]))

print('\nRuby - 3v: \n')
for i in range(len(ruby3v_ps)): 
    print('P%s - %.3f GPa' % (i,ruby3_ps[i]))

print('\nRuby - 4v: \n')
for i in range(len(ruby4v_ps)): 
    print('P%s - %.3f GPa' % (i,ruby4_ps[i]))
    
print('\nRuby - 5v: \n')
for i in range(len(ruby5v_ps)): 
    print('P%s - %.3f GPa' % (i,ruby5_ps[i]))

# 7. Plot the data.

In [ ]:
x = list(range(0,50))
plt.plot(x, ruby1_ps, 'k-', label='ruby1h')
plt.plot(x, ruby2_ps, 'b-', label='ruby2h')
plt.plot(x, ruby3_ps, 'r-', label='ruby3h')
plt.plot(x, ruby4_ps, 'g-', label='ruby4h')
plt.plot(x, ruby5_ps, 'm-', label='ruby5h')

plt.plot(x, ruby1v_ps, color='gray', label='ruby1v')
plt.plot(x, ruby2v_ps, color='slateblue', label='ruby2v')
plt.plot(x, ruby3v_ps, color='pink', label='ruby3v')
plt.plot(x, ruby4v_ps, color='palegreen', label='ruby4v')
plt.plot(x, ruby5v_ps, color='plum', label='ruby5v')

plt.axhline(y=max(ruby1_ps),color='k',linestyle='--')
plt.axhline(y=min(ruby1_ps),color='k',linestyle='--')
plt.axhline(y=max(ruby2_ps),color='b',linestyle='--')
plt.axhline(y=min(ruby2_ps),color='b',linestyle='--')
plt.axhline(y=max(ruby3_ps),color='r',linestyle='--')
plt.axhline(y=min(ruby3_ps),color='r',linestyle='--')
plt.axhline(y=max(ruby4_ps),color='g',linestyle='--')
plt.axhline(y=min(ruby4_ps),color='g',linestyle='--')
plt.axhline(y=max(ruby5_ps),color='m',linestyle='--')
plt.axhline(y=min(ruby5_ps),color='m',linestyle='--')

plt.axhline(y=max(ruby1v_ps),color='gray',linestyle='--')
plt.axhline(y=min(ruby1v_ps),color='gray',linestyle='--')
plt.axhline(y=max(ruby2v_ps),color='slateblue',linestyle='--')
plt.axhline(y=min(ruby2v_ps),color='slateblue',linestyle='--')
plt.axhline(y=max(ruby3v_ps),color='pink',linestyle='--')
plt.axhline(y=min(ruby3v_ps),color='pink',linestyle='--')
plt.axhline(y=max(ruby4v_ps),color='palegreen',linestyle='--')
plt.axhline(y=min(ruby4v_ps),color='palegreen',linestyle='--')
plt.axhline(y=max(ruby5v_ps),color='plum',linestyle='--')
plt.axhline(y=min(ruby5v_ps),color='plum',linestyle='--')

plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
plt.savefig('ruby-all-plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby1_ps, 'k-', label='ruby1 horizontal')
plt.axhline(y=max(ruby1_ps),color='k',linestyle='--')
plt.axhline(y=min(ruby1_ps),color='k',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby1v_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby1v_ps, color='gray', label='ruby1 vertical')
plt.axhline(y=max(ruby1v_ps),color='gray',linestyle='--')
plt.axhline(y=min(ruby1v_ps),color='gray',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby1v_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby2_ps, 'b-', label='ruby2 horizontal')
plt.axhline(y=max(ruby2_ps),color='b',linestyle='--')
plt.axhline(y=min(ruby2_ps),color='b',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby2_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby2v_ps, color='slateblue', label='ruby2 vertical')
plt.axhline(y=max(ruby2v_ps),color='slateblue',linestyle='--')
plt.axhline(y=min(ruby2v_ps),color='slateblue',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby2v_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby3_ps, 'r-', label='ruby3 horizontal')
plt.axhline(y=max(ruby3_ps),color='r',linestyle='--')
plt.axhline(y=min(ruby3_ps),color='r',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby3_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby3v_ps, color='pink', label='ruby3 vertical')
plt.axhline(y=max(ruby3v_ps),color='pink',linestyle='--')
plt.axhline(y=min(ruby3v_ps),color='pink',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby3v_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby4_ps, 'g-', label='ruby4 horizontal')
plt.axhline(y=max(ruby4_ps),color='g',linestyle='--')
plt.axhline(y=min(ruby4_ps),color='g',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby4_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby4v_ps, color='palegreen', label='ruby4 vertical')
plt.axhline(y=max(ruby4v_ps),color='palegreen',linestyle='--')
plt.axhline(y=min(ruby4v_ps),color='palegreen',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby4v_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby5_ps, 'm-', label='ruby5 horizontal')
plt.axhline(y=max(ruby5_ps),color='m',linestyle='--')
plt.axhline(y=min(ruby5_ps),color='m',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby5_plot.png',dpi=800)
plt.show()

In [ ]:
plt.plot(x, ruby5v_ps, color='plum', label='ruby5 vertical')
plt.axhline(y=max(ruby5v_ps),color='plum',linestyle='--')
plt.axhline(y=min(ruby5v_ps),color='plum',linestyle='--')
plt.xlabel('Position (distance)',fontweight='bold')
plt.ylabel('Pressure (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Pressure Variation" % (sample),fontweight='bold')
plt.legend()
plt.ylim(-0.03,0.24)
plt.xlim(0,49)
# plt.savefig('ruby5v_plot.png',dpi=800)
plt.show()

In [ ]:
print('Ruby Pressures (GPa):\n')

print('Ruby - 1_horizontal:')
print('MAX: %.4f' % max(ruby1_ps))
print('MIN: %.4f\n' % min(ruby1_ps))

print('Ruby - 1_vertical:')
print('MAX: %.4f' % max(ruby1v_ps))
print('MIN: %.4f\n' % min(ruby1v_ps))

print('Ruby - 2_horizontal:')
print('MAX: %.4f' % max(ruby2_ps))
print('MIN: %.4f\n' % min(ruby2_ps))

print('Ruby - 2_vertical:')
print('MAX: %.4f' % max(ruby2v_ps))
print('MIN: %.4f\n' % min(ruby2v_ps))

print('Ruby - 3_horizontal:')
print('MAX: %.4f' % max(ruby3_ps))
print('MIN: %.4f\n' % min(ruby3_ps))

print('Ruby - 3_vertical:')
print('MAX: %.4f' % max(ruby3v_ps))
print('MIN: %.4f\n' % min(ruby3v_ps))

print('Ruby - 4_horizontal:')
print('MAX: %.4f' % max(ruby4_ps))
print('MIN: %.4f\n' % min(ruby4_ps))

print('Ruby - 4_vertical:')
print('MAX: %.4f' % max(ruby4v_ps))
print('MIN: %.4f\n' % min(ruby4v_ps))

print('Ruby - 5_horizontal:')
print('MAX: %.4f' % max(ruby5_ps))
print('MIN: %.4f\n' % min(ruby5_ps))

print('Ruby - 5_vertical:')
print('MAX: %.4f' % max(ruby5v_ps))
print('MIN: %.4f\n' % min(ruby5v_ps))


pressures = [max(ruby1_ps),max(ruby2_ps),max(ruby3_ps),max(ruby4_ps),max(ruby5_ps),\
             max(ruby1v_ps),max(ruby2v_ps),max(ruby3v_ps),max(ruby4v_ps),max(ruby5v_ps),\
             min(ruby1_ps),min(ruby2_ps),min(ruby3_ps),min(ruby4_ps),min(ruby5_ps),\
             min(ruby1v_ps),min(ruby2v_ps),min(ruby3v_ps),min(ruby4v_ps),min(ruby5v_ps)]

print('OVERALL:')
print('MAX: %.4f' % (max(pressures)))
print('MIN: %.4f' % (min(pressures)))
print('Dif.: %.4f' % (max(pressures)-min(pressures)))